TEXT CLASSIFICATION USING BERT

In [ ]:
!pip install -q -U datasets transformers evaluate accelerate
!pip install -q huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 17.7 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/lib/python3.12/shutil.py", line 847, in move
    os.rename(src, real_dst)
OSError: [Errno 18] Invalid cross-device link: '/usr/local/lib/python3.12/dist-packages/pyarrow/' -> '/usr/local/lib/python3.12/dist-packages/~yarrow'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^

KeyboardInterrupt: 

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, TrainingArguments, Trainer

import numpy as np
import evaluate
import torch

In [ ]:
import wandb

wandb.init(mode="disabled")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


In [ ]:
ds = load_dataset("thainq107/ntc-scv")
ds

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/570 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.8M [00:00<?, ?B/s]

data/valid-00000-of-00001.parquet:   0%|          | 0.00/6.35M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/6.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'preprocessed_sentence'],
        num_rows: 30000
    })
    valid: Dataset({
        features: ['sentence', 'label', 'preprocessed_sentence'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['sentence', 'label', 'preprocessed_sentence'],
        num_rows: 10000
    })
})

In [ ]:
#tokenizer dataset
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True
    )
max_seq_len = 100
max_seq_len = min(max_seq_len, tokenizer.model_max_length)

def preprocess_data(texts):
    res = tokenizer(
        texts["preprocessed_sentence"], # Cột chứa văn bản đầu vào
        padding="max_length",           # Nếu câu ngắn hơn 100 từ -> Điền thêm số 0 cho đủ
        truncation=True,        	      # Nếu câu dài hơn 100 từ -> Cắt bớt phần đuôi
        max_length=max_seq_len          # Đặt ngưỡng là 100
    )
    res["label"] = texts["label"]
    return res

processed_dataset = ds.map(preprocess_data, batched=True)
processed_dataset

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'preprocessed_sentence', 'input_ids', 'attention_mask'],
        num_rows: 30000
    })
    valid: Dataset({
        features: ['sentence', 'label', 'preprocessed_sentence', 'input_ids', 'attention_mask'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['sentence', 'label', 'preprocessed_sentence', 'input_ids', 'attention_mask'],
        num_rows: 10000
    })
})

In [ ]:
#model
num_labels = 2
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
    finetuning_task="text-classification"
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=config,
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)

  return metric.compute(predictions=predictions, references=labels)

In [ ]:
#train
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end = True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["valid"],
    compute_metrics= compute_metrics,
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipython-input-641902166.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.437446,0.804600
2,No log,0.438518,0.801200
3,0.456300,0.413255,0.818800
4,0.456300,0.381425,0.837000
5,0.344900,0.396298,0.837600
6,0.344900,0.385771,0.838400
7,0.280200,0.400237,0.836000
8,0.280200,0.418403,0.842700
9,0.238100,0.433539,0.842300
10,0.238100,0.433134,0.840600


TrainOutput(global_step=2350, training_loss=0.3121680661465259, metrics={'train_runtime': 3041.1812, 'train_samples_per_second': 98.646, 'train_steps_per_second': 0.773, 'total_flos': 7761761640000000.0, 'train_loss': 0.3121680661465259, 'epoch': 10.0})

In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

In [ ]:
repo_id = "huynoob1312/my-finetuned-distilbert-base-uncased-ntc"

model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ..._p6b57m/model.safetensors:   0%|          |  575kB /  268MB            

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/huynoob1312/my-finetuned-distilbert-base-uncased-ntc/commit/da9f403fd5427b2f4d21e78e2938550019d569db', commit_message='Upload tokenizer', commit_description='', oid='da9f403fd5427b2f4d21e78e2938550019d569db', pr_url=None, repo_url=RepoUrl('https://huggingface.co/huynoob1312/my-finetuned-distilbert-base-uncased-ntc', endpoint='https://huggingface.co', repo_type='model', repo_id='huynoob1312/my-finetuned-distilbert-base-uncased-ntc'), pr_revision=None, pr_num=None)